In [1]:
# Add the facets overview python code to the python path
import sys
sys.path.append('./python')

# Load UCI census train and test data into dataframes.
import pandas as pd

In [2]:
import pyodbc

##cnxn = pyodbc.connect('Driver={SQL Server Native Client 11.0};Server=lasr-sqldb-prd-mss,17001;Database=MSS_U_NADBI_S;Trusted_Connection=yes')
##train_data=pd.read_sql("select top 1000 ParentShareClassName,INVSR_TYP_CD,CreditedParentOrgID from dbo.NADIA_DG_TRANSFORMED_PDFS_SCORE_FINAL",cnxn)

cnxn = pyodbc.connect('Driver={SQL Server Native Client 11.0};Server=lasr-sqldb-prd-mss,17001;Database=MSS_S_3PD_NS;Trusted_Connection=yes')
##train_data=pd.read_sql("select top 100000 AFAUM,ClearFirmID,IndustryAUM,Month,Quarter,Year,ClearFirmName,FirmType,MorningstarCategory,OfficeCountry from dbo.broadridge_bdg_ms_history_CURRENT",cnxn)
train_data=pd.read_sql("select entity_id,entity_resolution_status,month,platform_id,year,entity_match_status,entity_type,fund_name from dbo.BrightScope_CAPITAL_GROUP_AUM_CURRENT",cnxn)
test_data=pd.read_sql("select month,year,fund_name from dbo.BrightScope_CAPITAL_GROUP_AUM",cnxn)


In [3]:
# Calculate the feature statistics proto from the datasets and stringify it for use in facets overview
from generic_feature_statistics_generator import GenericFeatureStatisticsGenerator
import base64

gfsg = GenericFeatureStatisticsGenerator()
proto = gfsg.ProtoFromDataFrames([{'name': 'train', 'table': train_data},
                                  {'name': 'test', 'table': test_data}])
protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")

In [4]:
# Display the facets overview visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html" >
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display(HTML(html))